In [ ]:
# !pip install flask-ngrok
# !pip install flask_restful
# !pip install -U scikit-learn
# !pip install keras-facenet
# !pip install face_recognition

from flask import Flask
from flask_restful import Api, Resource
from bson.json_util import dumps
import requests
import os
from google.colab import drive
from flask_ngrok import run_with_ngrok
import numpy as np
from keras_facenet import FaceNet
import cv2
from skimage import io
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Input, Dense, Conv2D, MaxPool2D, UpSampling2D, Dropout, Flatten
from keras.models import Model
from keras.models import Sequential
from keras import backend as K
import keras
import os
import pickle
from matplotlib import pyplot as plt
from PIL import Image
import cv2
import math

#### CONSTs

IMG_SRC = "https://drive.google.com/uc?id="
ENT_THRESH = 0.5
CONTROLLER_DOMAIN = "https://71f1273f0fad.ngrok.io"
model = None
####functions

def face_net_encoder(classes, embedder):

    image_encodings = []
    tot_cnt = 0
    label_encodings = []
    count_list = []
    Images = []
    Y = []
    temp = 0
    for i in classes:
      #print(len(classes[i]))
      for j in range(len(os.listdir("/content/drive/My Drive/driver_classified/" + i))):
        Y.append(temp)
      temp = temp + 1

    Y = np.array(Y)
    
    for driver_class in classes:
          count = 0
          count_1 = 0
          for image in ((os.listdir("/content/drive/My Drive/driver_classified/" + driver_class))):
                detection = embedder.extract("/content/drive/My Drive/driver_classified/" + driver_class + "/" + image, threshold=0.95)
                if len(detection) != 0:# and count_1 < 20:
                    image_encodings.append(detection[0]['embedding'])
                    label_encodings.append(Y[tot_cnt])
                    Images.append(cv2.imread("/content/drive/My Drive/driver_classified/" + driver_class + "/" + image))
                    tot_cnt = tot_cnt + 1
                    count_1 = count_1 + 1
                else:
                    tot_cnt = tot_cnt + 1
                count_list.append(count_1)
    print(Y)
    return image_encodings, label_encodings

def one_hot_encoder(arr, enc):
  arr = np.array(arr)
  arr = enc.fit_transform(arr.reshape(-1,1)).toarray()
  return arr
  
def model_factory(class_count, X_train):
  n_cols = X_train.shape[1]
  model = Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(n_cols,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(class_count, activation= 'softmax', input_shape=(n_cols,))
  ])
  #model.add(Dense(128, activation = 'relu'))
  #model.add(Dropout(0.5))
  #model.add(Dense(64, activation = 'relu'))
  #model.add(Dropout(0.5))
  #opt = keras.optimizers.Adam(learning_rate = 0.0005)
  
  model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])
  model.summary()

  return model

def active_learner_factory(classes):

    # the pre-requisities for active learner are the following : x_train, y_train (viz => one_hot_label_encodings), model and label_encodings
    x_train = []
    y_train = []
    label_encodings = []
    one_hot_label_encodings = []
    image_encodings = []
    model = None

    embedder = FaceNet() # embedder for getting image encodings
    enc = OneHotEncoder(handle_unknown='ignore') #one hot encoder 

    image_encodings, label_encodings = face_net_encoder(classes, embedder) # here we get the label_encodings
    one_hot_label_encodings = one_hot_encoder(label_encodings, enc)
    x_train, y_train = image_encodings, one_hot_label_encodings # here we get the x_train & y_train
    x_train = np.array(x_train)
    model =  model_factory(len(classes),x_train)

    return x_train, y_train, label_encodings, model  

#function to get image encodings
def get_image_encoding(image):
  embedder = FaceNet()
  encoding = embedder.extract(image, threshold=0.95)
  if(encoding):
    return encoding[0]["embedding"]
  return []

def entropy(thing):
    ent = 0
    
    for i in thing:
        if i == 0:
            i = i + 0.00001
        ent = ent + i * math.log(i, 2)  
    return -ent
    
def activeLearningClassifier(image_encoding, model):
    ent_thresh = ENT_THRESH
    prediction = model.predict(image_encoding.reshape((-1, 512)))
    print(prediction)
    ent = entropy(prediction[0])
    if ent > ent_thresh:
        return "unknown", ent, prediction
    else:
        return np.argmax(prediction[0]),ent, prediction

def set_image_encodings(img_ids, X_train, Y_train, enc):

    for img_id in img_ids:
        image = io.imread(IMG_SRC + img_id)
        print(image)
        encoding = get_image_encoding(image)
        if len(encoding) > 0:
            X_train = np.concatenate((X_train,np.array(encoding).reshape(1,512)))

            img_id_arr= []
            img_id_arr.append(int(img_ids[img_id]))
            img_id_arr = np.array(img_id_arr)
            Y_train = np.concatenate((Y_train,img_id_arr))

    Y_train = one_hot_encoder(Y_train, enc)
    return X_train, Y_train
####

#RESOURCES
class GetClass(Resource):
    def __init__(self, model): #Here model is taken from kwargs
      self.model = model #Initialize model
    
    def post(self,img_id):
      source = IMG_SRC + img_id
      image = io.imread(source)

      print(str(model))
      # #TEMPORARY WORKAROUND for passing path instead of image
      # io.imsave("/content/drive/MyDrive/driver_classified/temp/"+img_id+".png",image)
      # #image_path = "/content/drive/MyDrive/driver_classified/temp/" + img_id + ".png"
      # image_list = os.listdir("/content/drive/My Drive/driver_classified/temp/")
      # image_path = image_list[0]
      # print(image_path)

      image_encoding = get_image_encoding(image)
      image_encoding = np.array(image_encoding)
      classification,ent, prediction = activeLearningClassifier(image_encoding,model)

      if classification=="unknown":
        response= requests.post(CONTROLLER_DOMAIN + "/copyimagebyid/" + img_id)


      result = {}
      result["classification"]=str(classification)
      result["ent"]=str(ent)
      result["prediction"]=str(prediction)
      return result

class Retrain(Resource):
    def __init__(self, **kwargs): #Here model is taken from kwargs
        # self.model = kwargs['model'] #Initialize model
        self.x_train = kwargs['x_train']
        self.y_train = kwargs['y_train']

    def post(self):
      response = requests.get(CONTROLLER_DOMAIN + "/getannotation")
      annotations = response.json()
      
      enc = OneHotEncoder(handle_unknown='ignore')
      print(self.x_train)
      self.x_train, self.y_train = set_image_encodings(annotations,self.x_train,self.y_train,enc)
      global model
      model.fit(self.x_train,self.y_train,epochs=10,verbose=1)

      return {'data':'SUCESSFULLY RETRAINED'}


####

#create server
if __name__=="__main__":

  app = Flask(__name__)
  api=Api(app)
  run_with_ngrok(app)  # Start ngrok when app is run

  drive.mount('/content/drive') #mount drive for images needed for cold start

  classes = (os.listdir("/content/drive/My Drive/driver_classified"))

  x_train, y_train, label_encodings, model = active_learner_factory(classes)
  
  #x_train, y_train, label_encodings, model = X_TRAIN,Y_TRAIN,LABEL_ENCODINGS,MODEL
  model.fit(x_train,y_train,epochs=10,verbose=1)
  print(x_train)

  #ADD RESOURCE MAPPINGS
  api.add_resource(GetClass,"/getclass/<string:img_id>",resource_class_kwargs={'model': model} )
  api.add_resource(Retrain,"/retrain",resource_class_kwargs={'x_train': x_train , 'y_train':label_encodings })
  app.run()
  # for / root, return Hello World
  # @app.route("/")
  # def root():
  #     print("here")
  #     print(response) 






Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: ignored

In [ ]:
X_TRAIN,Y_TRAIN,LABEL_ENCODINGS,MODEL = x_train, y_train, label_encodings, model

In [ ]:
model.predict(x_train)

array([[0.32945287, 0.3362174 , 0.33432972],
       [0.32330188, 0.33763543, 0.3390627 ],
       [0.32883146, 0.33462352, 0.33654502],
       [0.3300141 , 0.33758515, 0.3324008 ],
       [0.32270497, 0.34220466, 0.33509034],
       [0.3297068 , 0.33038157, 0.3399116 ],
       [0.32603025, 0.32944092, 0.3445288 ],
       [0.3340586 , 0.33475968, 0.33118165],
       [0.3349034 , 0.3392768 , 0.32581976],
       [0.3165922 , 0.33528396, 0.3481239 ],
       [0.3305127 , 0.33379436, 0.33569294],
       [0.3237646 , 0.34459656, 0.33163884],
       [0.33234447, 0.33545786, 0.3321977 ],
       [0.326296  , 0.3351513 , 0.33855268],
       [0.32732907, 0.3381481 , 0.33452278],
       [0.3121571 , 0.35140556, 0.33643734],
       [0.3281194 , 0.33187982, 0.34000075],
       [0.31692624, 0.34695557, 0.33611816],
       [0.32108045, 0.34187323, 0.33704633],
       [0.3157788 , 0.34409955, 0.3401217 ],
       [0.3309576 , 0.336003  , 0.33303937],
       [0.32966366, 0.32957995, 0.3407564 ],
       [0.

In [ ]:
ngrok.kill()
publ_url = ngrok.connect("8080")
print(publ_url)

In [ ]:
print(label_encodings)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
np.shape(x_train[0])

(512,)